In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import transforms, utils, models
from torch.utils.data import Dataset, DataLoader

In [4]:
from synthetic_utils import *

In [5]:
if torch.cuda.is_available():
    device="cuda"
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    device="cpu"
    torch.set_default_tensor_type('torch.FloatTensor')
print(device)

cpu


In [6]:
DATA_PATH = 'generated_data.pkl'

In [7]:
with open(DATA_PATH, 'rb') as f:
    imgs, shapes, colors = pickle.load(f)

In [8]:
imgs.shape, shapes.shape, colors.shape

((50000, 96, 96, 3), (50000, 1), (50000, 1))

In [9]:
imgs = imgs/255.
shapes = shapes.squeeze()
colors = colors.squeeze()

In [10]:
rs = np.random.RandomState(1791387)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
imgs_train, imgs_test, shapes_train, shapes_test, colors_train, colors_test = train_test_split(
    imgs, shapes, colors, test_size=0.1, stratify=shapes, random_state=rs,
)

In [13]:
imgs_train, imgs_val, shapes_train, shapes_val, colors_train, colors_val = train_test_split(
    imgs_train, shapes_train, colors_train, test_size=0.1, stratify=shapes_train, random_state=rs)

In [14]:
imgs_train.shape, imgs_val.shape, imgs_test.shape, shapes_train.shape, shapes_val.shape, shapes_test.shape, colors_train.shape, colors_val.shape, colors_test.shape

((40500, 96, 96, 3),
 (4500, 96, 96, 3),
 (5000, 96, 96, 3),
 (40500,),
 (4500,),
 (5000,),
 (40500,),
 (4500,),
 (5000,))

In [17]:
from sys import getsizeof

In [19]:
getsizeof(imgs)

11059200144

In [18]:
getsizeof(imgs_train)

8957952144

In [16]:
with open('gen_imgs.pkl', 'wb') as f:
    pickle.dump([imgs_train, imgs_val, imgs_test], f)

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [ ]:
with open('gen_val.pkl', 'wb') as f:
    pickle.dump([imgs_val, shapes_val, colors_val], f)
    
with open('gen_test.pkl', 'wb') as f:
    pickle.dump([imgs_test, shapes_test, colors_test], f)

In [ ]:
print('done')

In [ ]:
encoder = LeNet()
classifier = ClassNet()

In [ ]:
criterion = nn.BCELoss()
opt_cls = optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999))
opt_enc = optim.Adam(encoder.parameters(), lr=0.001, betas=(0.9, 0.999))

In [ ]:
losses, acc = train_encoder_classifier_epoch(encoder, classifier, imgs_train, shapes_train, opt_enc, opt_cls, criterion, device)
losses, acc

In [ ]:
val_loss, val_acc = validate_encoder_classifier_epoch(encoder, classifier,imgs_test, shapes_test, criterion, device)
val_loss, val_acc

In [ ]:
num_epochs = 1

In [ ]:
train_losses = []
val_losses = []
epoch_time = AverageMeter()
ep_end = time.time()
for epoch in range(0, num_epochs):
        print('Epoch: {}/{}'.format(epoch, start_epoch + num_epochs))
        scheduler_cls.step()
        scheduler_enc.step()
        # train
        train_loss, train_acc = train_encoder_classifier_epoch(encoder,
                                classifier, imgs_train, shapes_train, opt_enc,
                                opt_cls, criterion, device)
        train_losses.append(train_loss)
        # validate
        print('-'*10)
        val_loss, val_acc = validate_encoder_classifier_epoch(encoder, classifier,
                                imgs_test, shapes_test, criterion, device)
        print(val_loss)
        print(val_acc)

        print('Avg validation loss: {}'.format(val_loss))
        val_losses.append(val_loss)
        print('-' * 20)
        epoch_time.update(time.time() - ep_end)
        ep_end = time.time()
        print('Epoch {}/{}\t'
              'Time {epoch_time.val:.3f} sec ({epoch_time.avg:.3f} sec)'.format(epoch, start_epoch + num_epochs - 1, epoch_time=epoch_time))
        print('-'*20)